In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

hugging_face_api_key = os.environ["HUGGINGFACEHUB_API_TOKEN"]

In [2]:
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_chroma import Chroma


embed_model = SentenceTransformerEmbeddings(model_name="BAAI/bge-small-en-v1.5")

C:\Users\VARUN ARORA\AppData\Local\Temp\ipykernel_12672\1929084229.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embed_model = SentenceTransformerEmbeddings(model_name="BAAI/bge-small-en-v1.5")
d:\VS Code\python\GenAI-Cookbook\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
# Build a sample vectorDB
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load blog post
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(data)

vectordb = Chroma.from_documents(documents=splits, embedding=embed_model)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [17]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", temperature=0.7, )

question = "What are the approaches to Task Decomposition?"
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(), llm=llm
)

In [18]:
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [19]:
unique_docs = retriever_from_llm.invoke(question)
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['## Alternative Questions for "What are the approaches to Task Decomposition?":', '1. **Focusing on the purpose:** What are the different methods used to break down complex tasks into smaller, manageable subtasks, and what are the advantages and disadvantages of each approach?', '2. **Emphasizing specific contexts:** How can tasks be decomposed in different domains like software engineering, project management, or problem-solving?  ', '3. **Broadening the scope:** What are the key considerations when choosing a task decomposition approach, and how does the chosen approach impact task efficiency, collaboration, and overall project success?']


5

In [20]:
unique_docs

[Document(metadata={'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:', 'language': 'en', 'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log"}, page_content='Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.'),
 Document(metadata={'description': 'Building agents with LLM (

In [22]:
retriever_from_llm.invoke("What indicators does India and Nepal have in common?")

INFO:langchain.retrievers.multi_query:Generated queries: ['Here are three different versions of the user question, each focusing on a different aspect of the query:', '1. **Focusing on economic indicators:** What are the key economic indicators that both India and Nepal share, such as GDP growth rate, inflation, or unemployment rate?', '2. **Focusing on social indicators:** What social indicators are similar between India and Nepal, such as literacy rate, life expectancy, or access to healthcare?', '3. **Focusing on geopolitical indicators:** What geopolitical indicators do India and Nepal share, such as membership in regional organizations, shared borders, or historical ties?']


[Document(metadata={'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:', 'language': 'en', 'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log"}, page_content='(2) Model selection: LLM distributes the tasks to expert models, where the request is framed as a multiple-choice question. LLM is presented with a list of models to choose from. Due to the limited context length, task type based filtration is needed.\nInstruction:'),
 Document(metadata={'description': 'Building agent